Algorytm Wspinaczki HC
Paweł Brodziak
Magdalena Leśniak
Marceli Ptak

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import random
import time
import warnings
from enum import Enum
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
TSP_29 = pd.read_excel('TSP_29.xlsx')
TSP_29.name = 'TSP_29'
TSP_76 = pd.read_excel('TSP_76.xlsx')
TSP_76.name = 'TSP_76'
TSP_127 = pd.read_excel('TSP_127.xlsx')
TSP_127.name = 'TSP_127'
for df in [TSP_29, TSP_76, TSP_127]:
    df.index = np.arange(1, len(df) + 1)

In [3]:
class KryteriumStopu(Enum):
    iteracje = 'maksymalna liczba iteracji'
    max_bez_poprawy = 'maksymalna liczba iteracji bez poprawy'

##Oblicz odleglości między miastami i całkowitą odległość dla zadanej kolejności
def zmierz_odleglosci(miasta,df):
    odleglosci = []
    for i in range(len(miasta)-1):
        x = df[miasta[i]].iloc[miasta[i+1]-1]
        odleglosci.append(x)
        i+=1
    odleglosci.append(df[miasta[len(df)-1]].iloc[miasta[0]-1])
    suma_odleglosci = sum(odleglosci)
    return odleglosci, suma_odleglosci

##
def znajdz_sasiedztwo(miasta, suma_odleglosci,df):
    ob_odleglosc = suma_odleglosci
    nowe_miasta = miasta
    for i in range(len(miasta)):
        for j in range(i+1,len(miasta)):
            if i == j :
                continue;
            temp = np.copy(miasta)
            temp[i], temp[j] = temp[j], temp[i].copy()
            temp_odleglosci, temp_odleglosc = zmierz_odleglosci(temp,df)
            if temp_odleglosc < ob_odleglosc:
                nowe_miasta = temp
                ob_odleglosc = temp_odleglosc
    return nowe_miasta

def wspinaczka(df, kryterium_stopu:KryteriumStopu, max_iteracje=10000, max_iteracje_bez_poprawy=1000):
    miasta = [i for i in range(1,len(df)+1)]
    random.shuffle(miasta) ##losowa kolejność miast - wartość początkowa

    odleglosci, suma_odleglosci = zmierz_odleglosci(miasta,df)
    miasta_poczatkowe = miasta
    wartosc_poczatkowa = suma_odleglosci

    iteracje_bez_poprawy = 0
    iteracje_z_poprawa = 0
    iteracje = 0
    climb = True

    while climb == True:
        miasta_alt = znajdz_sasiedztwo(miasta,suma_odleglosci,df)
        odleglosci_alt, suma_odleglosci_alt = zmierz_odleglosci(miasta_alt,df)
        if suma_odleglosci_alt < suma_odleglosci:
            miasta = miasta_alt
            odleglosci = odleglosci_alt
            suma_odleglosci = suma_odleglosci_alt
            iteracje_bez_poprawy = 0
            iteracje_z_poprawa += 1
        elif suma_odleglosci_alt == suma_odleglosci:
            ## w przypadku takich samych wynikow, nowa kolejność wybierana jest losowo, lecz nie liczy się jako poprawa
            if random.randint(0,1) == 1:
                miasta = miasta_alt
                odleglosci = odleglosci_alt
                suma_odleglosci = suma_odleglosci_alt
                iteracje_bez_poprawy +=1
            else:
                iteracje_bez_poprawy +=1
        else:
            iteracje_bez_poprawy +=1

        iteracje += 1

        if kryterium_stopu == KryteriumStopu.iteracje and iteracje == max_iteracje:
            climb = False
        if kryterium_stopu == KryteriumStopu.max_bez_poprawy and iteracje_bez_poprawy == max_iteracje_bez_poprawy:
            climb = False

    return miasta, odleglosci, suma_odleglosci, iteracje_z_poprawa, miasta_poczatkowe, wartosc_poczatkowa

def wspinaczka_podsumowanie(df, kryterium_stopu:KryteriumStopu = KryteriumStopu.iteracje, max_iteracje=10000, max_iteracje_bez_poprawy=1000):
    start_time = time.time()
    miasta, odleglosci, suma_odleglosci, iteracje_poprawne, miasta_poczatkowe, wartosc_poczatkowa = wspinaczka(df, kryterium_stopu, max_iteracje,max_iteracje_bez_poprawy)
    end_time = time.time()
    execution_time = end_time - start_time
    if kryterium_stopu == KryteriumStopu.iteracje:
        iteracje = max_iteracje
    else:
        iteracje = max_iteracje_bez_poprawy
    print(f'''
    Kryterium stopu: {kryterium_stopu.value}
    liczba iteracji: {iteracje}
    liczba iteracji z poprawą: {iteracje_poprawne}
    Czas obliczeń: {execution_time:.2f} sekund
    wynik - suma odległości: {suma_odleglosci}
    kolejność miast: {miasta}
    Wartością początkową (odległością wylosowanych miast) była {wartosc_poczatkowa}
    ''')

In [4]:
## Algorytm wspinaczkowy dla danych TSP_29 z liczbą iteracji jako kryterium stopu i maksymalną liczbą iteracji 900000
wspinaczka_podsumowanie(TSP_29,kryterium_stopu=KryteriumStopu.iteracje,max_iteracje=300)


    Kryterium stopu: maksymalna liczba iteracji
    liczba iteracji: 300
    liczba iteracji z poprawą: 18
    Czas obliczeń: 37.03 sekund
    wynik - suma odległości: 2557
    kolejność miast: [20  4 15 18 17 10 12  9  5 21 13 19 14 22 11 25  7 23  8 27 16 24  1 28
  6 26 29  3  2]
    Wartością początkową (odległością wylosowanych miast) była 5735
    


In [5]:
## Algorytm wspinaczkowy dla danych TSP_29 z liczbą iteracji bez poprawy jako kryterium stopu i maksymalną liczbą takich iteracji 100
wspinaczka_podsumowanie(TSP_29,kryterium_stopu=KryteriumStopu.max_bez_poprawy,max_iteracje_bez_poprawy=50)


    Kryterium stopu: maksymalna liczba iteracji bez poprawy
    liczba iteracji: 50
    liczba iteracji z poprawą: 25
    Czas obliczeń: 7.71 sekund
    wynik - suma odległości: 2376
    kolejność miast: [ 7 25 19  2  3 29 26  9 12  6  1 24 16 13 10  4 15 11 22 14 17 18 20 21
  5 28  8 27 23]
    Wartością początkową (odległością wylosowanych miast) była 5487
    


In [6]:
## Algorytm wspinaczkowy dla danych TSP_127 z liczbą iteracji jako kryterium stopu i maksymalną liczbą iteracji wynoszącą milion
wspinaczka_podsumowanie(TSP_127,kryterium_stopu=KryteriumStopu.iteracje,max_iteracje=500)


    Kryterium stopu: maksymalna liczba iteracji
    liczba iteracji: 1000
    liczba iteracji z poprawą: 133
    Czas obliczeń: 10396.62 sekund
    wynik - suma odległości: 174390.4119282115
    kolejność miast: [109  96  88  87  86  85 104  92  89 125  59  19  21  17 105   7  50 121
   5 124  52 115  13 120   8  67 119  63 126  76  77  18  74  70  69  75
  81  84 117  78  20  15   1   2  51  57  56  47  49  53 118  46  48  33
 122  28  45  94 112 111 107 127  93 103  44  35  11   9 116  90   3  10
 114 106   6  24 108  12  14  41  43  34  39  30  31  79  80  82  83 102
 101  98  97 123  95  54  55  66  65  99 113  64  58   4  22  23  60  62
  61  91 100  16  37  36  40  42  38  25  29  32  26  27  72  73  68  71
 110]
    Wartością początkową (odległością wylosowanych miast) była 616053.9284768786
    


In [7]:
## Uruchomienie algorytmu wspinaczkowego z 100k iteracjami n razy w celu sprawdzenia średniego i najniższego wyniku a także średniego czasu w jakim dokonywane są obliczenia
i = 0
n = 100 ## liczba powtórzeń algorytmu - za każdym razem losowane jest nowe ustawienie początkowe
wszystkie_odleglosci = 0
czas_calkowity = 0
min_odleglosc = 999999
min_miasta = []
start_time_whole = time.time()
while i < n:
    start_time = time.time()
    miasta, odleglosci, suma_odleglosci, it_z_pop, miasta_poczatkowe, wartosc_poczatkowa = wspinaczka(TSP_29,kryterium_stopu=KryteriumStopu.max_bez_poprawy,max_iteracje_bez_poprawy=10)
    end_time = time.time()
    execution_time = end_time - start_time
    wszystkie_odleglosci += suma_odleglosci
    czas_calkowity += execution_time
    if suma_odleglosci < min_odleglosc:
        min_miasta = miasta
        min_odleglosc = suma_odleglosci
    else:
        pass
    i += 1

srednia_odleglosc = wszystkie_odleglosci / n
sredni_czas = czas_calkowity / n
end_time_whole = time.time()
overall_time = end_time_whole - start_time_whole
print(f'''
Po {n} powtórzeniach algorytmu iteracyjnego, najniższą zarejestrowaną odległością było {min_odleglosc}
Wynik był najlepszy dla poniższej kolejności miast
{min_miasta}

Średnio, podczas uruchomień algorytmu średnia odległość wynosiła {srednia_odleglosc}
Średni czas w jakim wykonywało się każde uruchomienie wszystkich iteracji wyniósł {sredni_czas:.3f} sekund.
Wszystkie algorytmy obliczone zostały w {overall_time:.3f} sekund ({overall_time / 60:.2f} minut).''')


Po 100 powtórzeniach algorytmu iteracyjnego, najniższą zarejestrowaną odległością było 2144
Wynik był najlepszy dla poniższej kolejności miast
[ 9  5 26 29  3  2 21  1  8 27 23  7 25 16 19 11 15 18 17 22 14  4 20 10
 13 24 28  6 12]

Średnio, podczas uruchomień algorytmu średnia odległość wynosiła 2480.53
Średni czas w jakim wykonywało się każde uruchomienie wszystkich iteracji wyniósł 3.216 sekund.
Wszystkie algorytmy obliczone zostały w 321.604 sekund (5.36 minut).
